In [312]:
import numpy as np
import matplotlib.pyplot as plt
from parse import parse
import cv2

In [313]:
train_images, train_labels = parse('digitdata/trainingimages', 'digitdata/traininglabels', True)
validation_images, validation_labels = parse('digitdata/validationimages', 'digitdata/validationlabels', True)
test_images, test_labels = parse('digitdata/testimages', 'digitdata/testlabels', True)

In [314]:
base_rate_true = np.bincount(train_labels)/train_labels.shape[0]
print(base_rate_true)

[0.0958 0.1126 0.0976 0.0986 0.107  0.0868 0.1002 0.11   0.0924 0.099 ]


In [315]:
weights = np.zeros((81,10))
bias = np.zeros(10)
kernel = np.ones((3,3))
images = np.swapaxes(np.swapaxes(train_images.astype(np.float64), 0, 2), 0, 1)
dst = np.zeros_like(images)

if images.shape[2] > 512:
    for i in range(int(np.ceil(images.shape[2]/512))):
        dst[:, :, i*512:(i+1)*512] = cv2.filter2D(src = images[:, :, i*512:(i+1)*512], ddepth = -1, kernel = kernel, anchor = (0,0))
else:
    dst= cv2.filter2D(src = images, ddepth = -1, kernel = kernel, anchor = (0,0))

zones = dst[::3,::3,:]


 

    

    

In [316]:
for k in range(100):
    for i in range(5000):
        b, index = 0, 0
        arr = zones[:,:,i].ravel()
        for j in range(10):
            a = np.dot(weights[:,j],arr) + bias[j]
            if a > b:
                b = a
                index = j
        if train_labels[i] != index:
            weights[:,index] -= arr
            bias[index] -= 1
            weights[:, train_labels[i]] += arr
            bias[train_labels[i]] += 1
      

       

In [317]:
tally = 0
vimages = np.swapaxes(np.swapaxes(validation_images.astype(np.float64), 0, 2), 0, 1)
vdst = np.zeros_like(vimages)

if vimages.shape[2] > 512:
    for i in range(int(np.ceil(vimages.shape[2]/512))):
        vdst[:, :, i*512:(i+1)*512] = cv2.filter2D(src = vimages[:, :, i*512:(i+1)*512], ddepth = -1, kernel = kernel, anchor = (0,0))
else:
    vdst= cv2.filter2D(src = images, ddepth = -1, kernel = kernel, anchor = (0,0))

vzones = vdst[::3,::3,:]


for i in range(1000):
    b, index = 0, 0
    arr = vzones[:,:,i].ravel()
    for j in range(10):
        a = np.dot(weights[:,j],arr) + bias[j]
        if a > b:
            b = a
            index = j
    if validation_labels[i] == index:
        tally += 1

print(tally/1000)
        

      
    



0.852
